In [2]:
import pickle
import numpy as np
import datasets
from datasets import load_dataset, Dataset
import pyarrow as pa
import pandas as pd


from dotenv import load_dotenv
load_dotenv()

import nltk


In [46]:
import sys
sys.path.append("../..")
from translate import translate_text


In [3]:
first_load = False
if first_load:
    gsm8k = load_dataset('gsm8k', 'main')
else:
    with open('data/multi_gsm8k.pickle', 'rb') as handle:
        gsm8k = pickle.load(handle)

In [4]:
gsm8k['train'][0]['question']

'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?'

In [4]:
gsm8k['train'][0]['answer']


'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'

In [7]:
gsm8k['test'][1]['question']

'A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?'

In [8]:
gsm8k['test'][0]['question']


"Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?"

In [9]:
gsm8k['test'][-1]['question']


'Henry and 3 of his friends order 7 pizzas for lunch. Each pizza is cut into 8 slices. If Henry and his friends want to share the pizzas equally, how many slices can each of them have?'

In [6]:
len(gsm8k['test'])


1319

In [9]:
gsm8k['test'][0]['answer']


'Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nShe makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.\n#### 18'

In [10]:
type(gsm8k['test'])

datasets.arrow_dataset.Dataset

In [12]:
len(gsm8k['test'])


1319

# Run translate function

In [45]:
def run_translate_dataset(lang_cod):
    ### call translate function
    test_question_gt = [translate_text(target=lang_cod, text=gsm8k['test'][i]['question']) for i in range(len(gsm8k['test']))]
    test_answer_gt = [translate_text(target=lang_cod, text=gsm8k['test'][i]['answer']) for i in range(len(gsm8k['test']))]
    gsm8k[f"test_{lang_cod}_gt"] = [{'question': question['translatedText'].replace("&lt;", "<").replace("&gt;", ">"),\
                            'answer': '\n'.join(nltk.sent_tokenize(answer['translatedText'].replace("&lt;", "<").replace("&gt;", ">")))}\
                            for question, answer in zip(test_question_gt, test_answer_gt)]
    
    ### convert to Huggingface dataset
    gsm8k[f"test_{lang_cod}_gt"] = Dataset(pa.Table.from_pandas(pd.DataFrame(gsm8k[f"test_{lang_cod}_gt"])))

    ### save to disk
    with open('data/multi_gsm8k.pickle', 'wb') as handle:
        pickle.dump(gsm8k, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Spanish

In [16]:
test_spanish_question_gt = [translate_text(target='es', text=gsm8k['test'][i]['question']) for i in range(len(gsm8k['test']))[:2]]
test_spanish_question_gt

[{'translatedText': 'Los patos de Janet ponen 16 huevos por día. Ella come tres para el desayuno cada mañana y hornea muffins para sus amigos todos los días con cuatro. Ella vende el resto en el mercado de granjeros diariamente a $2 por huevo de pato fresco. ¿Cuánto en dólares gana ella todos los días en el mercado de agricultores?',
  'input': "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?"},
 {'translatedText': 'Una túnica lleva 2 rollos de fibra azul y la mitad de fibra blanca. ¿Cuántos pernos se necesitan en total?',
  'input': 'A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?'}]

In [ ]:
run_translate_dataset(lang_cod='es')

## Portuguese


In [37]:
test_portuguese_question_gt = [translate_text(target='pt', text=gsm8k['test'][i]['question']) for i in range(len(gsm8k['test']))[:2]]
test_portuguese_question_gt

[{'translatedText': 'Os patos de Janet põem 16 ovos por dia. Ela come três no café da manhã todas as manhãs e faz muffins para seus amigos todos os dias com quatro. Ela vende o restante no mercado dos fazendeiros diariamente por $ 2 por ovo de pato fresco. Quanto em dólares ela ganha todos os dias no mercado dos fazendeiros?',
  'input': "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?"},
 {'translatedText': 'Um manto leva 2 parafusos de fibra azul e metade dessa quantidade de fibra branca. Quantos parafusos no total são necessários?',
  'input': 'A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?'}]

In [22]:
run_translate_dataset(lang_cod='pt')

## French

In [38]:
test_french_question_gt = [translate_text(target='fr', text=gsm8k['test'][i]['question']) for i in range(len(gsm8k['test']))[:2]]
test_french_question_gt

[{'translatedText': "Les canards de Janet pondent 16 œufs par jour. Elle en mange trois au petit-déjeuner tous les matins et prépare des muffins pour ses amis tous les jours avec quatre. Elle vend le reste au marché fermier tous les jours pour 2 $ l'œuf de cane frais. Combien gagne-t-elle en dollars chaque jour au marché fermier\xa0?",
  'input': "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?"},
 {'translatedText': 'Une robe prend 2 rouleaux de fibre bleue et la moitié de cette quantité de fibre blanche. Combien de boulons au total faut-il ?',
  'input': 'A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?'}]

In [39]:
run_translate_dataset(lang_cod='fr')

translated questions
translated answers


## Italian

In [29]:
test_italian_question_gt = [translate_text(target='it', text=gsm8k['test'][i]['question']) for i in range(len(gsm8k['test']))[:2]]
test_italian_question_gt

[{'translatedText': 'Le anatre di Janet depongono 16 uova al giorno. Ne mangia tre a colazione ogni mattina e prepara muffin per i suoi amici ogni giorno con quattro. Ogni giorno vende il resto al mercato degli agricoltori per $ 2 per uovo di anatra fresco. Quanto guadagna in dollari ogni giorno al mercato degli agricoltori?',
  'input': "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?"},
 {'translatedText': 'Una tunica prende 2 dardi di fibra blu e metà di quella fibra bianca. Quanti bulloni ci vogliono in totale?',
  'input': 'A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?'}]

In [43]:
run_translate_dataset(lang_cod='it')

translated questions
translated answers


# Test translations

In [32]:
gsm8k['test_es_gt'][0]

{'question': 'Los patos de Janet ponen 16 huevos por día. Ella come tres para el desayuno cada mañana y hornea muffins para sus amigos todos los días con cuatro. Ella vende el resto en el mercado de granjeros diariamente a $2 por huevo de pato fresco. ¿Cuánto en dólares gana ella todos los días en el mercado de agricultores?',
 'answer': 'Janet vende 16 - 3 - 4 = <<16-3-4=9>>9 huevos de pato al día.\nElla gana 9 * 2 = $<<9*2=18>>18 todos los días en el mercado de agricultores.\n#### 18'}

In [34]:
gsm8k['test_pt_gt'][0]


{'question': 'Os patos de Janet põem 16 ovos por dia. Ela come três no café da manhã todas as manhãs e faz muffins para seus amigos todos os dias com quatro. Ela vende o restante no mercado dos fazendeiros diariamente por $ 2 por ovo de pato fresco. Quanto em dólares ela ganha todos os dias no mercado dos fazendeiros?',
 'answer': 'Janet vende 16 - 3 - 4 = <<16-3-4=9>>9 ovos de pato por dia.\nEla faz 9 * 2 = $<<9*2=18>>18 todos os dias no mercado do fazendeiro.\n#### 18'}

In [35]:
gsm8k['test_fr_gt'][0]


{'question': 'Les canards de Janet pondent 16 œufs par jour. Elle en mange trois au petit-déjeuner tous les matins et prépare des muffins pour ses amis tous les jours avec quatre. Elle vend le reste au marché fermier tous les jours pour 2 $ l&#39;œuf de cane frais. Combien gagne-t-elle en dollars chaque jour au marché fermier\xa0?',
 'answer': 'Janet vend 16 - 3 - 4 = <<16-3-4=9>>9 œufs de cane par jour.\nElle gagne 9 * 2 = $<<9*2=18>>18 chaque jour au marché fermier.\n#### 18'}

In [43]:
gsm8k['test'][0]


{'question': "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?",
 'answer': 'Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nShe makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.\n#### 18'}

In [4]:
with open('data/multi_gsm8k.pickle', 'rb') as handle:
    gsm8k_p = pickle.load(handle)

In [5]:
gsm8k_p['test'][0]

{'question': "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?",
 'answer': 'Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nShe makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.\n#### 18'}

In [18]:
with open('data/multi_gsm8k.pickle', 'wb') as handle:
    pickle.dump(gsm8k_p, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [24]:
gsm8k_p['test_es_gt'][0]


{'question': 'Los patos de Janet ponen 16 huevos por día. Ella come tres para el desayuno cada mañana y hornea muffins para sus amigos todos los días con cuatro. Ella vende el resto en el mercado de granjeros diariamente a $2 por huevo de pato fresco. ¿Cuánto en dólares gana ella todos los días en el mercado de agricultores?',
 'answer': 'Janet vende 16 - 3 - 4 = <<16-3-4=9>>9 huevos de pato al día.\nElla gana 9 * 2 = $<<9*2=18>>18 todos los días en el mercado de agricultores.\n#### 18'}

In [26]:
gsm8k['test_es_gt'] = gsm8k_p['test_es_gt']
